In [ ]:

# COMPILAR DESDE GOOGLE COLAB
"""from google.colab import files
import pandas as pd

print("Selecciona archivo deptos_out.csv")
up = files.upload()
csv_path = list(up.keys())[0]

df = pd.read_csv(csv_path, encoding="utf-8-sig")"""


#COMPILAR DESDE LOCAL (EXTENSIÓN JUPYTER VSCODE)
import pandas as pd

# Cargar directamente desde la ruta local
df_basico = pd.read_csv('propiedades_portal_inmobiliario.csv', encoding='utf-8-sig')
df_detalle = pd.read_csv('propiedades_detalle_caracteristicas.csv', encoding='utf-8-sig')

# Merge por el link
df = pd.merge(df_basico, df_detalle, on='link', how='left')

print(f"Filas, Columnas: {df.shape}")
df.head(10)


In [ ]:
print("Filas, Columnas:", df.shape)
display(df.head(10))
print("\nColumnas:")
print(list(df.columns))

In [ ]:
import pandas as pd

cols_deseadas = ["precio","moneda","superficie_util","superficie_total","dormitorios","banos"]
cols_existentes = [c for c in cols_deseadas if c in df.columns]

df_modelo = df[cols_existentes].copy()

print("Columnas usadas:", cols_existentes)
print("Filas, Columnas:", df_modelo.shape)
display(df_modelo.head(10))

In [ ]:
# Dejar solo filas con moneda "$" y mostrar una muestra
df_pesos = df_modelo.copy()
df_pesos["moneda_norm"] = df_pesos["moneda"].astype(str).str.strip()

# nos quedamos estrictamente con "$"
df_pesos = df_pesos[df_pesos["moneda_norm"] == "$"].drop(columns=["moneda_norm"])

print("Filas, Columnas (solo $):", df_pesos.shape)
display(df_pesos.head(10))

In [ ]:
import re
import numpy as np

def to_m2_float(x):
    if pd.isna(x):
        return np.nan
    s = str(x).lower()
    s = s.replace("m²", "").replace("m2", "").replace("m^2", "")
    s = s.replace(",", ".")
    nums = re.findall(r"\d*\.?\d+", s)
    if not nums:
        return np.nan
    vals = [float(n) for n in nums if n != ""]
    if not vals:
        return np.nan
    # si hay rango (varios números), usamos el promedio
    return float(np.mean(vals))

for col in ["superficie_util", "superficie_total"]:
    if col in df_pesos.columns:
        df_pesos[col] = df_pesos[col].apply(to_m2_float)

print("Tipos de datos tras conversión:")
display(df_pesos.dtypes)

print("\nMuestra convertida:")
display(df_pesos.head(10))

In [ ]:
# solo por si acaso

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Partimos de df_pesos ya filtrado a "$" y con superficies en float
df_arriendo = df_pesos.copy()

# Asegurar tipos numéricos en columnas clave
for col in ["dormitorios", "banos", "precio"]:
    if col in df_arriendo.columns:
        df_arriendo[col] = pd.to_numeric(df_arriendo[col], errors="coerce")

# Completar una superficie con la otra usando ratio mediana
if {"superficie_util","superficie_total"}.issubset(df_arriendo.columns):
    ambos = df_arriendo[["superficie_util","superficie_total"]].dropna()
    ratio_mediana = (ambos["superficie_total"] / ambos["superficie_util"]).clip(0.6, 2.5).median() if len(ambos) >= 30 else 1.15
    m_util_nan = df_arriendo["superficie_util"].isna() & df_arriendo["superficie_total"].notna()
    df_arriendo.loc[m_util_nan, "superficie_util"] = df_arriendo.loc[m_util_nan, "superficie_total"] / ratio_mediana
    m_tot_nan  = df_arriendo["superficie_total"].isna() & df_arriendo["superficie_util"].notna()
    df_arriendo.loc[m_tot_nan,  "superficie_total"] = df_arriendo.loc[m_tot_nan,  "superficie_util"] * ratio_mediana

# Features y target para el modelo
features = [c for c in ["superficie_util","superficie_total","dormitorios","banos"] if c in df_arriendo.columns]
target = "precio"

# Conjunto entrenable con precio presente
df_trainable = df_arriendo[features + [target]].copy()
df_trainable = df_trainable[df_trainable[target].notna()]

# Imputación simple de features (necesaria para el modelo)
imp = SimpleImputer(strategy="median")
X_all = imp.fit_transform(df_trainable[features].values)
y_all = df_trainable[target].values

print("Features usadas:", features)
print("Shape entrenable:", df_trainable.shape)
display(df_trainable.head(10))


In [ ]:
# imports necesarios para el entrenamiento/metricas
!pip -q install scikit-learn

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

if len(features) == 0:
    raise ValueError("No hay features disponibles.")
if df_trainable.shape[0] < 50:
    print(f"Advertencia: pocas filas entrenables ({df_trainable.shape[0]}).")

In [ ]:
# Split
X_tr, X_va, y_tr, y_va = train_test_split(X_all, y_all, test_size=0.25, random_state=42)

# Objetivo logarítmico para estabilizar varianza
y_tr_log = np.log1p(y_tr)

# Modelo
rf = RandomForestRegressor(
    n_estimators=600, random_state=42, n_jobs=-1, min_samples_leaf=2
)
rf.fit(X_tr, y_tr_log)

# Validación en escala original ($)
y_va_pred_log = rf.predict(X_va)
y_va_pred = np.expm1(y_va_pred_log)

mae  = mean_absolute_error(y_va, y_va_pred)
rmse = np.sqrt(mean_squared_error(y_va, y_va_pred))
r2   = r2_score(y_va, y_va_pred)

print(f"Métricas validación ($):  MAE={mae:,.0f} | RMSE={rmse:,.0f} | R²={r2:.3f}")

# VALIDACION

In [ ]:
import numpy as np

mediana = float(np.median(y_va))
mape = float(np.mean(np.abs((y_va - y_va_pred) / np.maximum(y_va, 1e-6))) * 100)

print(f"Mediana precio valid: $ {mediana:,.0f}")
print(f"MAPE: {mape:.2f}%  (error relativo promedio)")

In [ ]:
import matplotlib.pyplot as plt
resid = y_va - y_va_pred

plt.figure(figsize=(5,5))
plt.scatter(y_va, y_va_pred, alpha=0.5)
mn, mx = min(y_va.min(), y_va_pred.min()), max(y_va.max(), y_va_pred.max())
plt.plot([mn,mx],[mn,mx],'--')
plt.xlabel("Precio real ($)")
plt.ylabel("Precio predicho ($)")
plt.title("Parity plot (valid)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
plt.hist(resid, bins=30)
plt.title("Residuales (y - ŷ)")
plt.xlabel("Error ($)")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import r2_score
y_tr_pred = np.expm1(rf.predict(X_tr))
print(f"R² train: {r2_score(y_tr, y_tr_pred):.3f}  |  R² valid: {r2:.3f}")

# MAPA DE CALOR GEOGRÁFICO

In [ ]:
# Agregar columna de comuna extrayendo del título
import re

def extraer_comuna(titulo):
    if pd.isna(titulo):
        return None
    titulo = str(titulo).lower()
    
    # Comunas de Santiago (principales)
    comunas = [
        'providencia', 'las condes', 'vitacura', 'ñuñoa', 'la reina',
        'santiago', 'san miguel', 'maipú', 'pudahuel', 'peñalolén',
        'la florida', 'macul', 'independencia', 'recoleta', 'conchalí',
        'quinta normal', 'estación central', 'cerrillos', 'lo prado',
        'huechuraba', 'lo barnechea', 'puente alto', 'la cisterna',
        'san joaquín', 'el bosque', 'pedro aguirre cerda', 'lo espejo',
        'san ramón', 'san bernardo', 'la granja', 'la pintana',
        'cerro navia', 'renca', 'quilicura', 'colina', 'lampa'
    ]
    
    for comuna in comunas:
        if comuna in titulo:
            return comuna.title()
    return 'Otra'

df_arriendo['comuna'] = df_arriendo.index.map(lambda i: extraer_comuna(df.loc[i, 'titulo']) if i in df.index else None)

# Calcular precio promedio por comuna
precio_comuna = df_arriendo.groupby('comuna')['precio'].mean().sort_values(ascending=False)
print("Precio promedio por comuna (CLP):")
print(precio_comuna.head(15))

In [ ]:
# Mapa de calor con folium
import folium
from folium.plugins import HeatMap

# Coordenadas aproximadas de comunas de Santiago (latitud, longitud)
coordenadas_comunas = {
    'Providencia': (-33.4372, -70.6106),
    'Las Condes': (-33.4166, -70.5833),
    'Vitacura': (-33.3833, -70.5833),
    'Ñuñoa': (-33.4569, -70.5978),
    'La Reina': (-33.4500, -70.5333),
    'Santiago': (-33.4372, -70.6506),
    'San Miguel': (-33.4969, -70.6533),
    'Maipú': (-33.5108, -70.7578),
    'Pudahuel': (-33.4403, -70.7500),
    'Peñalolén': (-33.4969, -70.5167),
    'La Florida': (-33.5242, -70.5989),
    'Macul': (-33.4914, -70.5978),
    'Independencia': (-33.4167, -70.6667),
    'Recoleta': (-33.4167, -70.6333),
    'Conchalí': (-33.3833, -70.6667),
    'Quinta Normal': (-33.4333, -70.7000),
    'Estación Central': (-33.4667, -70.6833),
    'Cerrillos': (-33.4972, -70.7167),
    'Lo Prado': (-33.4500, -70.7333),
    'Huechuraba': (-33.3667, -70.6333),
    'Lo Barnechea': (-33.3500, -70.5167),
    'Puente Alto': (-33.6167, -70.5833),
    'La Cisterna': (-33.5333, -70.6667),
    'San Joaquín': (-33.4969, -70.6322),
}

# Crear mapa centrado en Santiago
mapa = folium.Map(location=[-33.4489, -70.6693], zoom_start=11)

# Preparar datos para heatmap (lat, lon, precio normalizado)
heat_data = []
for comuna, precio_promedio in precio_comuna.items():
    if comuna in coordenadas_comunas and not pd.isna(precio_promedio):
        lat, lon = coordenadas_comunas[comuna]
        # Normalizar precio para intensidad del mapa (0-1)
        intensidad = min(precio_promedio / precio_comuna.max(), 1.0)
        heat_data.append([lat, lon, intensidad])
        
        # Agregar marcador con precio
        folium.CircleMarker(
            location=[lat, lon],
            radius=8,
            popup=f"{comuna}<br>Precio promedio: ${precio_promedio:,.0f}",
            color='red',
            fill=True,
            fillColor='red',
            fillOpacity=0.6
        ).add_to(mapa)

# Agregar capa de calor
HeatMap(heat_data, radius=25, blur=35, gradient={0.4: 'blue', 0.65: 'lime', 0.8: 'orange', 1.0: 'red'}).add_to(mapa)

# Guardar mapa
mapa.save('mapa_precios_santiago.html')
print("Mapa guardado en 'mapa_precios_santiago.html'")
mapa

# VALIDACION

In [ ]:
# Reentrenar con todo el set disponible
rf_full = RandomForestRegressor(
    n_estimators=600, random_state=42, n_jobs=-1, min_samples_leaf=2
)
y_all_log = np.log1p(y_all)
rf_full.fit(imp.fit_transform(df_trainable[features].values), y_all_log)

# Construimos el ejemplo:
sup_util = 140.0
# Si también usamos superficie_total, la estimamos con ratio_mediana
sup_total = sup_util * ratio_mediana if "superficie_total" in features else None

ejemplo = {}
for f in features:
    if f == "superficie_util":   ejemplo[f] = sup_util
    elif f == "superficie_total":ejemplo[f] = sup_total
    elif f == "dormitorios":     ejemplo[f] = 4
    elif f == "banos":           ejemplo[f] = 2

X_ex = imp.transform(pd.DataFrame([ejemplo], columns=features).values)
pred_log = rf_full.predict(X_ex)[0]
pred_clp = float(np.expm1(pred_log))

print("Consulta (arriendo): 4 dormitorios, 2 baños, 140 m² útil")
print(f"Precio estimado: ~$ {pred_clp:,.0f} CLP")
print("\nVector usado:", ejemplo)